/Users/timcosemans/Documents/PROJ-LLM-Zoomcamp/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ENCODER  = "huggingface"
ENCODER_MODEL = "intfloat/multilingual-e5-small" 
LLM_MODEL = "llama3.2"
INDEX_NAME = "llm-doc"
import sys
sys.path.append('..')

In [3]:
#https://discuss.elastic.co/t/issue-connecting-python-to-elasticsearch-in-docker-environment/361507/2
es_client = Elasticsearch(
    hosts=["https://localhost:9200"],
    basic_auth=('elastic', os.getenv("ELASTIC_PASSWORD")),
    verify_certs=False,
    max_retries=30,
    retry_on_timeout=True,
    request_timeout=30,
)

ollama = Client(host='https://localhost:11434')


/Users/timcosemans/Documents/PROJ-LLM-Zoomcamp/venv/lib/python3.13/site-packages/elasticsearch/_sync/client/__init__.py:311: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [ ]:

vector_search_term = model.encode(search_term)

In [48]:
context = []

knn_query = {
    "field": "question_encoded",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}
query = {
    "match": {
      "question": {
        "query": search_term,
      }
    }
  }
res = es_client.search(index=INDEX_NAME, 
                       query=query, knn=knn_query, source=["text", "section", "question", "course"])
for hit in res['hits']['hits']:
        context.append(hit['_source'])

/Users/timcosemans/Documents/PROJ-LLM-Zoomcamp/venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [49]:
context[0]

{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?'}

In [ ]:



def setup_model(model_name):
    """Download and setup the Ollama model if not available."""
    try:
        ollama.show(f'{model_name}:latest')
    except:
        print(f"Downloading {model_name}...")
        ollama.pull(f'{model_name}:latest')

def generate_response(query: str, context: str) -> str:
    """Generate response using Ollama with retrieved context."""
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in context:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()


    response = ollama.generate(
        model=model_name,
        prompt=prompt,
        options={
            'temperature': 0.7,
            'top_p': 0.9,
            'max_tokens': 500
        }
    )

    return response['response']

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
https://collabnix.com/building-rag-applications-with-ollama-and-python-complete-2025-tutorial/

https://github.com/ollama/ollama-python?tab=readme-ov-file

https://www.elastic.co/docs/solutions/search/search-approaches

Full-text search uses bm25 on the text and query directly. 
Vector search can be based ond dense vectors (and knn to find similar vectors) or sparse vectors. 
Among sparse vector models, ES provides ELSER. This basically uses BERT encoding to generate synonyms based on sequences of words in the query and content. 
The n highest scoring words are taken as expansion tokens and mapped back to a sparse vector.

Hybrid search uses vector and full-text search together.
Semantic search provides managed workflows that use vector search under the hood.

https://www.elastic.co/docs/reference/elasticsearch/clients/python/examples
https://github.com/elastic/elasticsearch-labs/tree/main/notebooks/search

ELSER is most easy to use, but not available in the free version of ES.
Plan
- Use hugging face to create a dense representation of question field of documents
- Create mapping (https://www.elastic.co/docs/solutions/search/vector/knn)
- Create index 
